# Multithreading

This notebook demonstrates how you can do multithreading with Python. 

A good reason why you would like to do this, is when you have to do many asynchronous API calls and you have to wait for a callback with the response. Instead of doing this from a single thread and spending a lot of time waiting, multithreading gives you the option to send out multiple API calls at the same time. This way 5 threads, can give you a speed up of 5 times.

The concept of multithreading, and other parralellisation options (multiprocessing and asyncio) are greatly explained and visualized in [realpython blog](https://realpython.com/python-concurrency/). This notebook demonstrates the multithreading option only using `concurrent.futures.ThreadPoolExecutor()`.

A visualization of what's going on taken from the above blog:

<img src="https://files.realpython.com/media/Threading.3eef48da829e.png" width="600" align="left">



The nice thing compared to multithreading is that we are sharing the same memory space. This can also be tricky as multiple threads could be writing to the same memory at the same time, but that can be avoided using the global interpreter lock. In this notebook I will demonstrate how to do that by appending to the same output csv file from multiple threads.

A good explanation on the differences between multiprocessing and multithreading can be found in this [stackoverflow question](https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python).



## A Simple example

In this first example we just want to test if we can get the multithreading to work at all. For this purpose we simulate processing time by letting our function sleep for 3 seconds. We need this function to be executed 10 times, and given that theses 10 are not dependent of one another, we can thus use multithreading to speed it up.

Doing this in a single thread, we would expect 10 * 3 = 30 seconds of processing time. 

Having 5 threads at the same time, we expect to be done in 10 / 5 * 3 = 6 seconds.

Let's see if this is the case:

In [1]:
import concurrent.futures
import threading
import time


def sleep_some(item):
    time.sleep(SLEEP_SECONDS)
    print(f"Thread {threading.get_ident()} ready!")


def process_all(some_iterable):
    with concurrent.futures.ThreadPoolExecutor(max_workers=NR_THREADS) as executor:
        executor.map(sleep_some, some_iterable)

In [2]:
SLEEP_SECONDS = 3
NR_THREADS = 5
NR_ITEMS = 10

start_time = time.time()
process_all(range(NR_ITEMS))
duration = time.time() - start_time
print(f"Processed {NR_ITEMS} in {duration} seconds")

Thread 123145594302464 ready!Thread 123145610067968 ready!

Thread 123145599557632 ready!Thread 123145589047296 ready!
Thread 123145604812800 ready!

Thread 123145594302464 ready!Thread 123145610067968 ready!Thread 123145599557632 ready!Thread 123145604812800 ready!


Thread 123145589047296 ready!

Processed 10 in 6.010974168777466 seconds


Yep! We indeed got our expected result of 6 seconds, plus a little bit of overhead. 

Note that we used `threading.get_ident()` to print the thread number. From this we get our confidence that we have indeed run our function in 5 different threads, and that that each thread processed 2 items from our list.

## How to pass arguments to the parallelized function

Sometimes you want to provide an additional argument to the function that you are parallelizing. For example, if you are making an API call you might want to pass on an api key that you don't want to hardcode in your function. 

Adding such an argument can be done in 3 ways, as explained neatly [here](https://yuanjiang.space/threadpoolexecutor-map-method-with-multiple-parameters). Below demonstrates the way using `itertools`'s `repeat()` function, where we will parameterize the sleep seconds argument.

In [3]:
from itertools import repeat


def sleep_some(item, sleep_seconds):
    time.sleep(sleep_seconds)
    print(f"Thread {threading.get_ident()} ready!")
    
def process_all(some_iterable, sleep_seconds):
    with concurrent.futures.ThreadPoolExecutor(max_workers=NR_THREADS) as executor:
        executor.map(sleep_some, some_iterable, repeat(sleep_seconds))

In [4]:
SLEEP_SECONDS = 2

start_time = time.time()
process_all(range(NR_ITEMS), SLEEP_SECONDS)
duration = time.time() - start_time
print(f"Processed {NR_ITEMS} in {duration} seconds")

Thread 123145589047296 ready!Thread 123145594302464 ready!Thread 123145604812800 ready!Thread 123145610067968 ready!Thread 123145599557632 ready!




Thread 123145594302464 ready!Thread 123145604812800 ready!Thread 123145589047296 ready!Thread 123145599557632 ready!Thread 123145610067968 ready!




Processed 10 in 4.013882875442505 seconds


Great! The program has done what we want by passing on the new value for `SLEEP_SECONDS` as a parameter.

## How to (safely) write to the same file

At this point we can simply replace the `sleep_some()` function with an asynchronous API call, and we know how to pass on parameters to it. Last thing that remains is how to gather the API responses from each thread into one file. 

For this, we will use the `threading.Lock()` functionality as is suggested [here](https://stackoverflow.com/questions/39797340/python-multthreading-safe-to-use-pandas-to-csv-on-common-file). It works very easily. Just wrap your write call in it to protect the write operation while the other threads still run in parallel. 

```python
csv_output_lock = threading.Lock()

with csv_output_lock:
    my_df.to_csv("results.csv",mode = 'a')
```

Adding it to our example, this is how we would write the ready statements of each thread in a csv file:

In [5]:
output_lock = threading.Lock()


def sleep_some(item, sleep_seconds):
    time.sleep(sleep_seconds)
    with output_lock:
        with open('multithreading_example.txt','a') as file:
            file.write(f"Thread {threading.get_ident()} ready! \n")

In [6]:
start_time = time.time()
process_all(range(NR_ITEMS), SLEEP_SECONDS)
duration = time.time() - start_time
print(f"Processed {NR_ITEMS} in {duration} seconds")

Processed 10 in 4.010267019271851 seconds


Done! Go and inspect your output file to verify that it is successful.

Note: as the lock prevented the 5 threads to write to the file at the same time, our procedure takes a bit longer. All 5 threads finish at the same time but only one can write to file, so the other 4 are idle until the one is finished. Luckily, this doesn't take long and we still experience a huge speed up :)